In [1]:
import requests
import pandas as pd
import numpy as np
import requests
import json

In [2]:
def get_guards(roomid,ruid,page=1):
    payload={}
    headers = {
      'Cookie': 'LIVE_BUVID=AUTO1616700756065396'
    }
    url = f"https://api.live.bilibili.com/guard/topList?roomid={roomid}&ruid={ruid}&page={page}"
    
    response = requests.request("GET", url, headers=headers, data=payload)
    rg = json.loads(response.text)
    
    result = []
    g_num = rg['data']['info']['num']
    g_page = rg['data']['info']['page']
    # print(f"Guards Num: {g_num}, Total page num: {g_page}")
    guard_attr = ['uid','ruid','username','guard_level']
    result += [{k: v for k, v in u.items() if k in guard_attr} for u in rg['data']['top3']]
    
    for p in range(page, g_page+1):
        # if p%10 == 0:
            # print(f"Processing Page {p*10}...")
        url = f"https://api.live.bilibili.com/guard/topList?roomid={roomid}&ruid={ruid}&page={p}"
        response = requests.request("GET", url, headers=headers, data=payload)
        rg = json.loads(response.text)
        userList = [{k: v for k, v in u.items() if k in guard_attr} for u in rg['data']['list']]
        result += userList
    
    df_g = pd.DataFrame.from_dict(result)
    return df_g

In [3]:
def get_fans(ruid,page=1,page_size=100):
    '''get members (active viewers) in fans group
        - ruid: mid of the user
        - page: start from page #
        - page_size: number of members per page (max 30 actually))

    return: pandas dataframe'''
    payload={}
    headers = {
      'Cookie': 'LIVE_BUVID=AUTO1616700756065396'
    }
    url = f"https://api.live.bilibili.com/xlive/general-interface/v1/rank/getFansMembersRank?ruid={ruid}&page={page}&page_size={page_size}"
    
    response = requests.request("GET", url, headers=headers, data=payload)
    rf = json.loads(response.text)
    
    ### Get total number of this type of streamers
    fan_attr = ['user_rank','uid','name','score','medal_name','level','guard_level']
    f_num = rf['data']['num']
    f_page = int(f_num/30) + 1
    # print(f"Fans Num: {f_num}, Total page num: {f_page}")
    
    
    ### Get data from each page
    result = []
    for p in range(page, f_page+1):
        # if p%10 == 0:
            # print(f"Processing Page {p}...")
        url = f"https://api.live.bilibili.com/xlive/general-interface/v1/rank/getFansMembersRank?ruid={ruid}&page={p}&page_size={page_size}"
        response = requests.request("GET", url, headers=headers, data=payload)
        rf = json.loads(response.text)
        userList = [{k: v for k, v in u.items() if k in fan_attr} for u in rf['data']['item']]
        result += userList
    
    df_f = pd.DataFrame.from_dict(result)
    return df_f

In [13]:
source = pd.read_csv("data/streamer_info.csv")
source.shape

(577, 7)

In [21]:
## For each streamer, get their active viewers 
def fans_main(df, start=0):
    for index, row in df.iloc[start:].iterrows():
        print(f"Processing {index} vTuber {row['uname']}...")
        df = get_fans(row['mid']) 
        df.to_csv(f"data/fans/{row['uname']}_fans.csv", index=False)
    print("Fans data. Done!")

In [22]:
fans_main(df=source, start=575)

Processing 575 vTuber 桀骜文风...
Processing 576 vTuber 美波七海-official...
Fans data. Done!
